In [18]:
import numpy as np
import nltk
import random
import json
import torch
import sys
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from nltk.stem.porter import PorterStemmer
import speech_recognition as sr
import pyttsx3
#user defined modules
import Face_exp_model  #face emotion recognition
import speech_recognition_model #speech emotion recognition
from model import NeuralNet
from nltk_utils import tokenize,stem,bag_of_words #tokenize the sentence using nltk
from model_training import model_training # chatbot model training
print("completed")

completed


### Face expression recogintion and Speech erm 

In [12]:
face_output = Face_exp_model.Facerecognization()
print("completed")

1/1 [==============================] - 0s 27ms/step
completed


In [13]:
# Set up the speech recognition object
r = sr.Recognizer()

# Set up the microphone stream
mic = sr.Microphone()

# Set up the text-to-speech engine
engine = pyttsx3.init()

text = "talk something to detect your emotion by speech"
    # Speak the response
engine.say(text)
engine.runAndWait()

speech_recognition_model.recordAudio()
speech_output = speech_recognition_model.Speechrecognition()

Recording
Finished recording
1/1 [==============================] - 0s 444ms/step


In [14]:
print(face_output)
print(speech_output)

happy
disgust


In [15]:
model_training()

202 patterns
28 tags: ['Amusement', 'Sarcasm', 'accomplishment', 'appreciating_the_present_moment', 'celebration', 'compliment', 'excitement', 'experiencing_new_things', 'funny', 'good_news', 'goodbye', 'gratitude', 'greeting', 'guitar', 'happy', 'helping', 'interests', 'job_success', 'neutral', 'neutral_smalltalk', 'new_experience', 'no_particular_events', 'relaxation', 'relaxing_or_taking_a_break', 'sad', 'sadness_General', 'sadness_Loss', 'thanks']
268 unique stemmed words: ["'m", "'re", "'s", "'ve", ',', 'a', 'about', 'accomplish', 'adventur', 'all', 'am', 'amaz', 'an', 'and', 'ani', 'anxiou', 'anyth', 'appreci', 'are', 'assist', 'at', 'away', 'awesom', 'bad', 'balanc', 'be', 'beauti', 'been', 'befor', 'beginn', 'believ', 'best', 'better', 'boredom', 'brainstorm', 'break', 'burnt', 'bye', 'ca', 'can', 'care', 'celebr', 'comfort', 'compani', 'congratul', 'could', 'crack', 'day', 'depress', 'destress', 'did', 'die', 'do', 'done', 'down', 'downtim', 'dream', 'easi', 'easier', 'elat', 

'training complete. file saved to data.pth'

## chatbot

In [16]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)
    FILE = "C:\\Users\\gana4\\MAJOR PROJECT\\data.pth"
    data = torch.load(FILE)

    input_size = data["input_size"]
    hidden_size = data["hidden_size"]
    output_size = data["output_size"]
    all_words = data['all_words']
    tags = data['tags']
    model_state = data["model_state"]

    model = NeuralNet(input_size, hidden_size, output_size).to(device)
    model.load_state_dict(model_state)
    model.eval()

    bot_name = "chatbot"
x = face_output
y = speech_output
def combination_emotion():
    
    if (x == "happy" or x == "neutral" or x == "surprise") and (y == "neutral" or y == "calm" or y == "happy" or y == "surprised"):
        emotion = 'happy'
        return emotion
    elif (x == "happy" or x == "neutral" or x == "surprised") and (y == "sad" or y == "angry" or y == "fearful" or y == "disgust"):
        emotion = "neutral"
        return emotion
    elif (x == "angry" or x == "fear" or x == "sad") and (y == "neutral" or y == "calm" or y == "happy" or y == "surprised"):
        emotion = "neutral"
        return emotion
    elif (x == "angry" or x == "fear" or x == "sad") and (y == "sad" or y == "angry" or y == "fearful" or y == "disgust"):
        emotion = "sad"
        return emotion
    # print("Let's chat! (type 'quit' to exit)")
def chatbot():
    #initiative
    x = "How can i call you?"

    with mic as source:
        r.adjust_for_ambient_noise(source)
        print("chatbot :"+x)
        def handle_input(audio):
            try:
                u_name = r.recognize_google(audio)
                print("you: "+u_name)
                if 'is' in u_name: 
                    u_name = u_name.split("is")[-1].strip()
                    user_name = u_name
                elif 'am' in u_name:
                    u_name = u_name.split("am")[-1].strip()
                    user_name = u_name
                elif 'me' in u_name:
                    u_name = u_name.split("me")[-1].strip()
                    user_name = u_name
                elif 'as' in u_name:
                    u_name = u_name.split("as")[-1].strip()
                    user_name = u_name
                else:
                    user_name =  u_name

                emotion = combination_emotion()
                if emotion == "happy":
                    text2 = f"chatbot: {user_name}, I'm glad to hear that! Tell me more about why you're feeling happy."
                elif emotion == "sad":
                    text2 = f"chatbot: {user_name}, I'm sorry to hear that. Would you like to talk more about what's been making you feel sad?"
                elif emotion == "angry":
                    text2 = f"chatbot: {user_name}, I can understand why you might be feeling angry. Would you like to talk about what's been frustrating you?"
                elif emotion == "fearful":
                    text2 = f"chatbot: {user_name}, I'm sorry to hear that you're feeling scared. Would you like to talk more about what's been worrying you?"
                else:
                    text2  = f"chatbot: {user_name}, It sounds like you're feeling neutral about things. Is there anything in particular that's been on your mind?"
                print(text2)    
            except sr.UnknownValueError:
                respond('Sorry, I could not understand what you said')
            except sr.RequestError as e:
                respond('Sorry, there was an error processing your request')
                
        while True:
            try:
                print("listening...")
                audio = r.listen(source, timeout=6.0)
                handle_input(audio)
                break
            except sr.WaitTimeoutError:
                print('No input detected for 6 seconds, ending session')
                respond('Goodbye')
                break
            except sr.UnknownValueError:
                respond('Sorry, I could not understand what you said')
                break
            except sr.RequestError as e:
                respond('Sorry, there was an error processing your request')
                break

        

def responses():
    while True:
        def respond(sentence):
            sentence = tokenize(sentence)
            X = bag_of_words(sentence, all_words)
            X = X.reshape(1, X.shape[0])
            X = torch.from_numpy(X).to(device)

            output = model(X)
            _, predicted = torch.max(output, dim=1)

            tag = tags[predicted.item()]

            probs = torch.softmax(output, dim=1)
            prob = probs[0][predicted.item()]
            if prob.item() > 0.75:
                for intent in intents['intents']:
                    if tag == intent["tag"]:
                        bot_chat = random.choice(intent['responses'])
                        print(f"{bot_name}: {bot_chat}")
            else:
                print(f"{bot_name}: I do not understand...")

        # Define a function to handle user input
        def handle_input(audio):
            try:
                sentence = r.recognize_google(audio)
                print(f"You: {sentence}")
                if sentence == "exit":
                    sys.exit()
                else:
                    respond(sentence)
                
            except sr.UnknownValueError:
                print("Sorry, I could not understand what you said repeat again")
                engine.say('Sorry, I could not understand what you said, repeat again')
                engine.runAndWait()
            except sr.RequestError as e:
                engine.say('Sorry, there was an error processing your request')
                engine.runAndWait()
            

        # Start the microphone stream
        with mic as source:
            r.adjust_for_ambient_noise(source)
            while True:
                try:
                    print("listening...")
                    audio = r.listen(source,timeout = 6.0)
                    handle_input(audio)
                except sr.WaitTimeoutError:
                    print('Sorry, I it is timed out')
                    break

In [ ]:
chatbot()
responses()

chatbot :How can i call you?
listening...
you: call me as Durga
chatbot: as Durga, It sounds like you're feeling neutral about things. Is there anything in particular that's been on your mind?
listening...
You: no nothing tell me a joke
chatbot: Until next time, stay well
listening...
Sorry, I could not understand what you said repeat again
listening...
You: tell me a joke hello
chatbot: What did the buffalo say when his son left for college? Bison.
listening...
You: spectrum 8257 DMA diagram
chatbot: I do not understand...
listening...
